<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/header.png' width=1200/>

In this notebook, we will demonstrate how **Legend** and **Morphir** models can be interpreted on **Databricks** with high governance standards and minimum development overhead. We will be sourcing information with Legend and feeding the resulting dataframe into a Morphir calculation that we can safefy exchange with counterparties using **Delta Sharing** protocol.

## LEGEND DELTA
With our Legend model packaged as JAR and included as a cluster dependency, we can easily access each of its underlying entities, create tables programmatically or execute queries according to legend specifications. **Released at OSSF 2021 as a Labs project, we are happy to announce that this [utility](https://github.com/finos/legend-community-delta) is now a FINOS top level project**.

<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/legend1.png' width=400/>
<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/legend2.png' width=400/>
<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/legend3.png' width=400/>

In [0]:
%scala
import org.finos.legend.spark.LegendClasspathLoader
val legend = LegendClasspathLoader.loadResources()
display(legend.getEntityNames.toList.toDF("legendEntity"))

legendEntity
lcr::entities::product
lcr::services::getInflows
lcr::lakehouse::finos
lcr::entities::collateralClass
lcr::services::getInflowsWithBuckets
lcr::lakehouse::assetMapping
lcr::entities::currency
lcr::entities::asset
lcr::lakehouse::store
lcr::lakehouse::databricks


A service defined on Legend can act as an implicit contract that can be programmatically interpreted and enforced as follows. In the below query, we will source all the required data and run some transformations expressed as legend functions in order to build a dataframe that comply with Morphir specifications. Note that we did not have to code any of that business logic (JOIN operations, user defined functions, etc.) since that was abstracted by our legend model and enforced by Delta Lake tables.

In [0]:
%scala
val inflows = legend.query("lcr::services::getInflowsWithBuckets")
display(inflows.limit(10))

product,subProduct,collateralClass,marketValue,maturityBucket,encumbranceType,forwardStartAmount,forwardStartBucket,treasuryControl
UnsettledAssetPurchases,Unsettled (Forward),e_3,4222800.0,16,encumbrance_1,null,null,true
ForwardAssetPurchases,firm long,e_4,3574080.0,35,encumbrance_2,null,null,true
EncumberedAssets,Customer long,e_5,685440.0,4,encumbrance_3,null,null,false
UnencumberedAssets,Specific central bank,e_6,1676880.0,23,encumbrance_4,1139544.0,29,true
Capacity,Currency and Coin,e_7,636480.0,20,encumbrance_1,null,null,true
UnrestrictedReserveBalances,Level 1,e_8,3635280.0,7,encumbrance_2,3226464.0,31,true
RestrictedReserveBalances,Level 2a,e_9,1921680.0,20,encumbrance_3,1544688.0,32,true
UnsettledAssetPurchases,Level 2b,e_10,1603440.0,16,encumbrance_4,null,null,true
ForwardAssetPurchases,Non-HQLA,n_1,2239920.0,25,encumbrance_1,null,null,false
EncumberedAssets,No Collateral Pledged,n_2,2007360.0,14,encumbrance_2,1559376.0,35,true


## MORPHIR ON SPARK
Morphir can transpile its business logic into its desired target state technology (such as spark). Using Apache Maven and the use of Archetypes, we created a simple template as a FINOS Labs project to not "just" transpile Morphir to Scala but compile the generated Scala code into a machine interpretable Jar file that can be injected here as a dependency through a simple CI/CD process.

<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/morphir1.png' width=400/>
<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/morphir2.png' width=400/>

In [0]:
%scala
import regulation.us.lcr.inflows.assets.{SparkJobs => Morphir}
val lcr = inflows.transform(Morphir.sumToRule)
display(lcr)

Label,value
20(c)(1),1.84183848E9
20(b)(1),9.2004408E8
20(a)(1),3.4855236E9


We persist our results back onto a delta table. That table can be shared across different systems or organizations or mapped back onto Legend to be accessed by end users and downstream services. Persisting our reports to Delta format, we benefit from its full audit capability allowing users to travel back in time through all its previous versions.

In [0]:
%scala
lcr
  .write
  .format("delta")
  .mode("overwrite")
  .saveAsTable("lcr.report")

In [0]:
%sql
DESCRIBE HISTORY lcr.report

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2022-12-02T15:07:43.000+0000,3806966946971215,ashley.trainor@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1202-145735-yzcf5pzh,1,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/11.3.x-photon-scala2.12
1,2022-12-02T15:07:20.000+0000,3806966946971215,ashley.trainor@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1202-145735-yzcf5pzh,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/11.3.x-photon-scala2.12
0,2022-12-02T15:06:45.000+0000,3806966946971215,ashley.trainor@databricks.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1562452343328116),1202-145735-yzcf5pzh,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 769)",null,Databricks-Runtime/11.3.x-photon-scala2.12


We access our report as it was generated at a given point in time (regardless of susequent updates) or at a given version. This ensures audit and compliance requirements and, combined with Legend and Morphir, guarantees strict reproducibility of data, model, calculations (i.e. our trade report).

In [0]:
%sql
SELECT * FROM lcr.report
TIMESTAMP AS OF '2022-12-02T15:07:29.000+0000'

Label,value
20(c)(1),1.84183848E9
20(b)(1),9.2004408E8
20(a)(1),3.4855236E9


## DELTA SHARING
Persisting data to Delta Lake also offers an endless possibility of downstream use cases and data consumption. Whilst data can be mapped back onto legend and accessed via services, we can easily expose our output to end users via delta sharing, allowing users to access our trade reports (or LCR calculations here) without having to physically transfer that information across to a different system.

<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/excel1.png' width=400/>
<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/excel2.png' width=400/>
<img src='https://raw.githubusercontent.com/aamend/ossf_db_ms_gs/main/images/excel3.png' width=400/>

As an open protocol, this allows technologists to access this information using the tool of choice, Java, Python, Spark and business users to access the same using popular BI frameworks (Tableau, PowerBI) or even **Excel**.

In [0]:
%sql
CREATE RECIPIENT feds;
CREATE SHARE lcr;
ALTER SHARE lcr ADD TABLE lcr.report;
GRANT SELECT ON SHARE lcr TO feds;